## denpendency


In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# import IPython.display as ipd

# import os
# import json
# import math
# import torch
# from torch import nn
# from torch.nn import functional as F
# from torch.utils.data import DataLoader

# import commons
# import utils
# from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
# from models import SynthesizerTrn
# from text.symbols import symbols
# from text import text_to_sequence

# 


# def get_text(text, hps):
#     text_norm = text_to_sequence(text, hps.data.text_cleaners)
#     if hps.data.add_blank:
#         text_norm = commons.intersperse(text_norm, 0)
#     text_norm = torch.LongTensor(text_norm)
#     return text_norm

## LJ Speech

In [13]:
from scipy.io.wavfile import write
import utils
single_cfg_path = "/workspaces/vits/model/miu_config.json"
hps = utils.get_hparams_from_file(single_cfg_path)
singe_ct_path = "/workspaces/vits/model/miu.pth"
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
from text import text_to_sequence
import commons


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [8]:
from models import SynthesizerTrn
from text.symbols import symbols
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).to(device)
_ = net_g.eval()

_ = utils.load_checkpoint(singe_ct_path, net_g, None)

In [15]:
import IPython.display as ipd
from scipy.io.wavfile import write
stn_tst = get_text("別に私の連れなんだから、気にするほどじゃないでしょう？", hps)
with torch.no_grad():
    x_tst = stn_tst.to(device).unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()

audio_name = "sad.wav"
write(audio_name,rate=22050,data=audio)
ipd.display(ipd.Audio(filename=audio_name))

## VCTK

In [ ]:
import utils
hps = utils.get_hparams_from_file("./configs/vctk_base.json")
mul_ct_path = ""
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from models import SynthesizerTrn
from text.symbols import symbols
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).to(device)
_ = net_g.eval()

_ = utils.load_checkpoint(mul_ct_path, net_g, None)

In [ ]:
from text import text_to_sequence
import commons


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
import IPython.display as ipd

sid =  torch.LongTensor([1]).to(device)
stn_tst = get_text("別に私の連れなんだから、気にするほどじゃないでしょう？", hps)
with torch.no_grad():
    x_tst = stn_tst.to(device).unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    # sid = torch.LongTensor([4]).to(device)
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

### Voice Conversion

In [ ]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.to(device) for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).to(device)
    sid_tgt2 = torch.LongTensor([2]).to(device)
    sid_tgt3 = torch.LongTensor([4]).to(device)
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))